In [1]:
import os

os.environ['JAVA_HOME'] = "C:/Program Files/Java/jdk-11"
os.environ['PYSPARK_PYTHON'] = "C:/Users/usr/anaconda3/envs/pyspark_env/python.exe"
os.environ['PYSPARK_DRIVER_PYTHON'] = "C:/Users/usr/anaconda3/envs/pyspark_env/python.exe"
os.environ['HADOOP_HOME'] = "C:/hadoop-3.4.0"
os.environ['HADOOP_COMMON_LIB_NATIVE_DIR'] = "C:/hadoop-3.4.0/lib/native"
os.environ['PATH'] += os.pathsep + "C:/hadoop-3.4.0/bin"

import findspark
findspark.init()

from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()

sc = spark.sparkContext

In [42]:
players = spark.read.csv("data/practice_spark_sql_avanzado/players.csv", header=True, inferSchema=True)
app = spark.read.csv("data/practice_spark_sql_avanzado/appearances.csv", header=True, inferSchema=True)
competicion = spark.read.csv("data/practice_spark_sql_avanzado/competitions.csv", header=True, inferSchema=True)
juegos = spark.read.csv("data/practice_spark_sql_avanzado/games.csv", header=True, inferSchema=True)

- **Determine los tres países con mayor número de jugadores(jugadores nacidos en ese país). El resultado debe estar ordenado de forma descendente.** 

In [43]:
players.show()

+---------+---------------+--------------------+--------------------+----------------+----------------------+-------------+----------+------------------+-----+------------+-------------------+---------------------------+--------------------+
|player_id|current_club_id|                name|         pretty_name|country_of_birth|country_of_citizenship|date_of_birth|  position|      sub_position| foot|height_in_cm|market_value_in_gbp|highest_market_value_in_gbp|                 url|
+---------+---------------+--------------------+--------------------+----------------+----------------------+-------------+----------+------------------+-----+------------+-------------------+---------------------------+--------------------+
|    38790|          28095|      dmitri-golubov|      Dmitri Golubov|           UdSSR|                Russia|   1985-06-24|    Attack|    Centre-Forward| Both|         178|               null|                   675000.0|https://www.trans...|
|   106539|          28095|  ale

In [44]:
from pyspark.sql.functions import col, desc

In [47]:
players.groupBy("country_of_birth").count().orderBy(desc("count")).filter(col('country_of_birth').isNotNull()).limit(3).show()

+----------------+-----+
|country_of_birth|count|
+----------------+-----+
|          France| 1694|
|           Spain| 1388|
|           Italy| 1312|
+----------------+-----+



- **Obtenga la lista de jugadores con tarjeta roja. La salida debe contener dos columnas, el nombre de pila del jugador y la cantidad de tarjetas rojas que tiene.**

In [50]:
(app.groupBy("player_id")
    .agg(sum(col('red_cards')).alias('total_rojas'))
    .join(players, ['player_id'], 'left')
    .select(col('pretty_name'), col('total_rojas'))
    .orderBy(col('total_rojas').desc())
).show()

+-----------------+-----------+
|      pretty_name|total_rojas|
+-----------------+-----------+
|       Thomas Lam|          5|
|  Mario Balotelli|          5|
|     Granit Xhaka|          5|
| Domenico Berardi|          5|
|           Hilton|          5|
|Kalidou Koulibaly|          4|
|  Yannick Cahuzac|          4|
|      Jaume Costa|          4|
|    Heung Min Son|          4|
|       David Luiz|          4|
|  Gabriel Paletta|          4|
|       Allan Nyom|          4|
|  Damien Da Silva|          4|
|  Rodrigo De Paul|          4|
|  Stefan Mitrovic|          4|
|      Ivan Ordets|          4|
|          Rodrigo|          4|
|       Ante Rebic|          4|
|  Roberto Soriano|          4|
|     Jordan Amavi|          4|
+-----------------+-----------+
only showing top 20 rows



- **¿Cuántos juegos se jugaron en la Premier League? La salida debe contener dos columnas, el nombre de la liga y la cantidad de juegos que se jugaron en ella.**

- **Obtenga las tres ligas con mayor número de asistencia de público teniendo en cuenta todos los juegos que se jugaron en ellas. El resultado debe estar ordenado de forma descendente y tener dos columnas, el nombre de la liga y la asistencia total.**